In [4]:
from bioTextAugPackage.init import *
from bioTextAugPackage import search_terms
import requests
import pandas as pd
import json
from tqdm import tqdm

UMLSKEY="23c9650b-e1b9-4932-8ca8-fd13ae45a13c"

In [ ]:
df = pd.read_csv("export.csv")
df.columns
snomed_umls = dict()
for index, row in df.iterrows():
    snomed_umls[row["n.id"]] = search_terms.get_cui_for_code(apikey=UMLSKEY, codes=[row["n.id"]], sabs='SNOMEDCT_US')
json.dump(snomed_umls, open("snomed2umls.json", "w"))

In [5]:
snomed_umls = json.load(open("snomed2umls.json"))
snomed_umls = {k:eval(v) for k,v in snomed_umls.items()}

GET SYNONYMS

In [6]:
uri = 'https://uts-ws.nlm.nih.gov'
synonyms = dict()
version = 'current'

for snomed_code, umls_codes in tqdm(snomed_umls.items()):
  synonyms[snomed_code] = []
  synonyms_all = []
  for umls_code in umls_codes:
    content_endpoint = '/rest/content/' + str(version) + '/CUI/' + str(umls_code)
    query = {'apiKey': UMLSKEY}
    r = requests.get(uri + content_endpoint, params=query)
    r.encoding = 'utf-8'

    if r.status_code != 200:
        continue

    items = r.json()
    json_data = items['result']
    atoms = json_data['atoms']

    pages = items['pageSize']
    for page in np.arange(pages):
        try:
            atom_query = {'apiKey': UMLSKEY, 'pageNumber': page}
            a = requests.get(atoms, params=atom_query)
            a.encoding = 'utf-8'

            if a.status_code != 200:
                continue
            all_atoms = a.json()
            json_atoms = all_atoms['result']
            for atom in json_atoms:
                    if atom['language']=='ENG':
                        synonyms_all.append(atom['name'].lower())
        except: ""
  synonyms[snomed_code] = list(set(synonyms_all))

100%|██████████| 1021/1021 [3:17:04<00:00, 11.58s/it] 


In [7]:
json.dump(synonyms, open("synonyms.json", "w"))

In [8]:
import csv
with open('dict.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, values in synonyms.items():
        for value in values:
            writer.writerow([key, value])